In [27]:
import rasterio
import cv2
import numpy as np
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import geopandas as gpd
import os

%matplotlib inline

In [2]:
raster_path = "/mnt/storage_4tb/ymi/geo_data/40m_for_ori/citrusuco/segmentation_draw_lr/rgb_bin_504.tif"

In [15]:
with rasterio.open(raster_path, "r") as src:
    mask = src.read(2)
    bin_meta= src.meta

In [21]:
def save_gpd_df(save_path, geom_list, meta):
    geom_df = gpd.GeoDataFrame(geometry=geom_list)
    geom_df.crs = meta.get('crs')
    if os.path.exists(save_path):
        os.remove(save_path)
    geom_df.to_file(save_path, driver='GeoJSON', encoding='utf-8')

In [5]:
def polygonize(contours, meta, transform=True):
    polygons = []
    for i in tqdm_notebook(range(len(contours))):
        c = contours[i]
        n_s = (c.shape[0], c.shape[2])
        if n_s[0] > 2:
            if transform:
                polys = [tuple(i) * meta['transform'] for i in c.reshape(n_s)]
            else:
                polys = [tuple(i) for i in c.reshape(n_s)]
            polygons.append(Polygon(polys))
    return polygons

In [16]:
mask = ((mask > 128) * 255).astype(np.uint8)

In [17]:
mask.dtype

dtype('uint8')

In [20]:

im2, contours, hierarchy = cv2.findContours(mask.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
polygons = polygonize(contours, bin_meta)

In [22]:
save_path = "/mnt/storage_4tb/ymi/geo_data/40m_for_ori/citrusuco/row_data/504/connected_polygons.geojson"

In [28]:
save_gpd_df(save_path, polygons, bin_meta)

/home/user/.virtualenvs/cv34/lib/python3.5/site-packages/geopandas/io/file.py:62: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
